<a href="https://colab.research.google.com/github/valmirf/mineracao_textual/blob/main/Classifica%C3%A7%C3%A3o/03a_Classifica%C3%A7%C3%A3oTextos_BoW_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizado Bayesiano: Naive Bayes

O algoritmo Naive Bayes, assim como o KNN é um algoritmo de aprendizado com implementação relativamente simples, e que pode levar a resultados muito bons em determinados problemas de classificação. Este consiste em aplicar o teorema de Bayes, com a prerrogativa de independencia condicional entre cada par de atributos dado o valor da classe.

Clique nos links para mais informações sobre o algoritmo [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes) e sobre [classificação de textos](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html). com scikit-learn.

# Exemplo: classificação de texto

Neste exemplo, utilizaremos um dataset chamado 20newsgroups. 

O conjunto de dados é uma coleção de aproximadamente 20.000 documentos de grupos de notícias, particionados (quase) uniformemente em 20 grupos/categorias de notícias diferentes. Mais informações sobre esta base podem ser obtidas no repositório [UCI](http://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups)

As categorias existentes são:

* 'alt.atheism',
* 'comp.graphics',
* 'comp.os.ms-windows.misc',
* 'comp.sys.ibm.pc.hardware',
* 'comp.sys.mac.hardware',
* 'comp.windows.x',
* 'misc.forsale',
* 'rec.autos',
* 'rec.motorcycles',
* 'rec.sport.baseball',
* 'rec.sport.hockey',
* 'sci.crypt',
* 'sci.electronics',
* 'sci.med',
* 'sci.space',
* 'soc.religion.christian',
* 'talk.politics.guns',
* 'talk.politics.mideast',
* 'talk.politics.misc',
* 'talk.religion.misc'

Neste exemplo, vamos considerar apenas duas categorias: '**alt.med**' e '**comp.graphics**'. O scitkit contém uma função que auxilia o download desta base:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# Categorias selecionadas
categories = [
    'comp.graphics',
    'sci.med',
]

print("Carregando 20 newsgroups dataset para as categorias:")
print(categories)

twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_test  = fetch_20newsgroups(subset='test',  categories=categories, shuffle=True, random_state=42)
print("%d documents" % len(twenty_train.filenames))
print("%d categories" % len(twenty_train.target_names))

In [ ]:
# Visualizar os dados coletados
print(twenty_train.keys())

In [ ]:
# Visualizando um dos documentos
print(twenty_train['data'][0])

In [ ]:
# tamanho (sem preprocessamento)
len(twenty_train['data'][0])

## Preprocessamento

Como etapa de préprocessamento, iremos apenas nos limitar a remover palavras muito comuns, as chamadas *stopwords*. Uma lista de possíveis stopwords para inglês encontra-se abaixo:

In [ ]:
stopwords = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at',
             'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by',  'can', "can't", 'cannot', 'could', "couldn't", 
             'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during','each', 'few', 'for', 'from', 'further', 
             'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's",
             'hers', 'herself', 'him', 'himself', 'his', 'how', "how's",'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 
             'it', "it's", 'its', 'itself','1st', '2nd', '3rd','4th', '5th', '6th', '7th', '8th', '9th', '10th'
             "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself','no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours' 'ourselves', 'out', 'over', 'own',
             'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 
             'than', 'that',"that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", 
             "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 
             'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
             "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's",'will', 'with', "won't", 'would', "wouldn't", 
             'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', 
             'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'hundred', 'thousand']

O código abaixo utiliza a biblioteca NLTK para fazer o pré-processamento dos textos:

In [ ]:
import string
from nltk.tokenize.regexp import RegexpTokenizer

def preprocess(text):
  
  # remover pontuações
  text   = text.translate(string.punctuation)
  
  # converter para lowercase
  text = text.lower()
  
  # tokenizar o texto em palavras
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(text.lower())

  # filtrar palavras
  tokens = [word for word in tokens
            if word not in stopwords       # descartar stopwords
                and len(word) > 3          # descartar palavras com menos de 3 caracteres
                and not word[0].isdigit()] # descartar tokens contendo apenas numeros

  return ' '.join(tokens)

X_train = []
for doc in twenty_train['data']:
  X_train.append(preprocess(doc))
  
X_test = []
for doc in twenty_test['data']:
  X_test.append(preprocess(doc))
  
print(X_train[0])

In [ ]:
# tamanho (com preprocessamento)
len(X_train[0])

### Exercício

Converter o a função de preprocessamento acima para utilizar o SpaCy

In [ ]:
# sua resposta

## Representação vetorial do texto

Neste exemplo, utilizaremos o algoritmo Naive Bayes para classificar documentos de texto em categorias. Para isso, precisamos antes converter o texto para uma representação vetorial, ou seja, cada documento/exemplo precisa ser representado por um vetor de dimensões pré-definidas. Utilizaremos três técnicas básicas: BOW (*Bag of Words*), TF (*Term Frequency*) e TF-IDF (*Term Frequency - Inverse Document Frequency*)

### BOW

Consiste basicamente em contar quantas vezes cada palavra aparece no documento. Ou seja, sua aplicação a um conjunto de *n* documentos, produz uma matriz *n x d*, onde *d* corresponde ao tamanho do vocabulário considerado. No Scikit, esta representação é implementada pelo [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer):

In [ ]:
# Exemplo de uso do BOW no scikit

from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.toarray())  

Aplicando ao nosso dataset:

In [ ]:
vectorizer = CountVectorizer()
bow_model  = vectorizer.fit(X_train)

X_bow_train = bow_model.transform(X_train)
X_bow_test  = bow_model.transform(X_test)

print(X_bow_train.shape,X_bow_test.shape)

In [ ]:
# matriz está armazenada em formato sparse
print(X_bow_train[0,:])

### *Term Frequency* (TF)
A contagem de ocorrências (i.e., BOW) é um bom começo, mas há um problema: documentos mais longos terão valores de contagem média mais altos do que documentos mais curtos, embora possam falar sobre os mesmos tópicos.

Para evitar essas possíveis discrepâncias, basta dividir o número de ocorrências de cada palavra em um documento pelo número total de palavras no documento:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=False)
tf_model = vectorizer.fit(X_train)

X_tf_train = tf_model.transform(X_train)
X_tf_test  = tf_model.transform(X_test)

print(X_tf_train[0,:])

### TF-IDF
Palavras que aparecem em muitos documentos podem não ser interessantes para diferencia a classe desses documentos. Portanto, podemos dar um peso maior as palavras que aparecem em poucos documentos, pois essas palavras funcionam melhor para diferenciar as classes entre os documentos. TF-IDF é uma indicação do poder de discriminação do termo. O Log é usado para diminuir o efeito em relação a tf.

Usando o scikit, basta ativar o flag `use_idf=True`:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True)
tfidf_model = vectorizer.fit(X_train)

X_tfidf_train = tfidf_model.transform(X_train)
X_tfidf_test  = tfidf_model.transform(X_test)

print(X_tfidf_train[0,:])

# Treinando modelo NaiveBayes (NB)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

## BOW + NB

In [ ]:
clf.fit(X_bow_train, twenty_train.target)

acc = clf.score(X_bow_test , twenty_test.target)
print('Acurácia: ', acc)

## TF + NB

In [ ]:
clf.fit(X_tf_train, twenty_train.target)

acc = clf.score(X_tf_test , twenty_test.target)
print('Acurácia: ', acc)

## TF-IDF + NB

In [ ]:
clf.fit(X_tfidf_train, twenty_train.target)

acc = clf.score(X_tfidf_test , twenty_test.target)
print('Acurácia: ', acc)

# Testando iterativamente

In [ ]:
docs = ['The patient is getting well', 'OpenGL on the GPU is fast']
preprocessed_docs = [preprocess(doc) for doc in docs]

print('\nafter preprocessing:')
print(preprocessed_docs)

docs_preds = clf.predict(tfidf_model.transform(preprocessed_docs))

print('\npredictions:')
for i,doc in enumerate(docs):
    print('{} -> {}'.format(doc, twenty_train.target_names[docs_preds[i]]))
    

## Classification Report

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

preds = clf.predict(X_tfidf_test)


#print(metrics.plot_confusion_matrix(clf, X_tfidf_test, twenty_test.target, 
#                                    display_labels=twenty_test.target_names,
#                                    normalize='true'))


print(metrics.classification_report(twenty_test.target, preds,
                                    target_names=twenty_test.target_names))


print('Matriz de Confusao:')
cm = confusion_matrix(twenty_test.target,preds)
f = sns.heatmap(cm, annot=True, fmt='d')
